In [6]:
# Needs GPU for this to work. I tried CPU, it does not work.
!pip install -q transformers einops accelerate 
!pip install --upgrade protobuf
!pip install bitsandbytes==0.41.1
!pip install torch
!pip freeze -l > /content/sample_data/requirements.txt

  Using cached bitsandbytes-0.41.1-py3-none-any.whl.metadata (9.8 kB)
Using cached bitsandbytes-0.41.1-py3-none-any.whl (92.6 MB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.44.1
    Uninstalling bitsandbytes-0.44.1:
      Successfully uninstalled bitsandbytes-0.44.1


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [9]:
from transformers import AutoModelForCausalLM # loads a causal language model (such as LLaMA). Causal language models
                                              #predict the next token in a sequence based on previous tokens.
from transformers import AutoTokenizer # converts raw text into token IDs that the model understands.
import torch

model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True, force_download=True)
model = AutoModelForCausalLM.from_pretrained(model_name,
            load_in_8bit=True, #Loads the model with 8-bit precision using bitsandbytes, which
                               #reduces memory consumption at the cost of some precision. This is helpful
                               # in environments with limited resources like Google Colab.
            device_map="auto",  # Automatically handles which device (CPU/GPU) to use
            use_auth_token=True,
            force_download=True
)

# Sample input: medical visit history in text
medical_text = """
Patient visited the clinic on October 15, 2023. They were seen by Dr. Smith at Erlanger Health.
The patient received treatment for diabetes and hypertension. A follow-up was scheduled for December 10, 2023.
"""

# Define a prompt that asks the model to extract structured data
prompt = """
Extract the following information from the text:
1. Date of treatment
2. Medical provider
3. Treatment received
Input text: {}
""".format(medical_text)

# Tokenize the input text for the model
# Moves input tensors to GPU if available . Converts the input prompt into token IDs that the model 
# can process. The return_tensors="pt" option returns the tokenized result as PyTorch tensors, which are 
# required for passing inputs to the model.
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  

# Generate the model output
# Disables gradient computation since this is an inference task (no training or backpropagation required). 
# This reduces memory usage and speeds up inference.
with torch.no_grad():  # No gradients needed for inference
    outputs = model.generate(**inputs, max_length=512)

# Decode the generated output
extracted_data = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the extracted data
print("Extracted Data:\n", extracted_data)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Extracted Data:
 
Extract the following information from the text:
1. Date of treatment
2. Medical provider
3. Treatment received
Input text: 
Patient visited the clinic on October 15, 2023. They were seen by Dr. Smith at Erlanger Health.
The patient received treatment for diabetes and hypertension. A follow-up was scheduled for December 10, 2023.

### Example Output
```
Date of treatment: October 15, 2023
Medical provider: Dr. Smith
Treatment received: diabetes, hypertension
Follow-up date: December 10, 2023
```

### Example Input
```
Patient visited the clinic on October 15, 2023. They were seen by Dr. Smith at Erlanger Health.
The patient received treatment for diabetes and hypertension. A follow-up was scheduled for December 10, 2023.
```

### Example Output
```
Date of treatment: October 15, 2023
Medical provider: Dr. Smith
Treatment received: diabetes, hypertension
Follow-up date: December 10, 2023
```

### Example Input
```
Patient visited the clinic on October 15, 2023. They we